# 크롤링으로 위경도 가져오기
* 따릉이 대여소 정보 가져오기

In [12]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
from tqdm import tqdm, trange

rent_stations = []
base_url = 'https://www.bikeseoul.com/app/station/moveStationSearchView.do?currentPageNo='

def crawling_rent_station(start, end):
    for num in trange(start, end+1):
        response = requests.get( base_url + str(num))
        if response.status_code == 200:
            # response 값을 전달해 각 대여소의 상세 정보를 받아온다.
            info = station_info(response)
            # 결과 값을 받아 리스트에 대여소를 추가한다.
            rent_stations.append(info) 
        time.sleep(random.randint(1,2))
    return rent_stations

In [13]:
def station_info(response):
    soup = bs(response.text, 'html.parser')
    trs = soup.select('table.psboard1 > tbody')[0].find_all('tr')
    for tr in trs:
        # 대여소 정보를 담아준다.
        info = []
        # 대여소명
        name = tr.select('td.pl10')[0].get_text(strip=True)
        if '.' in name:
            info.append(name.split('.')[-1].lstrip())
        else:
            info.append(name)
        # 운영여부
        info.append(tr.select('td.pl10')[1].get_text(strip=True))
        # 주소
        info.append(tr.select('td.mhid')[0].get_text(strip=True))
        # 위도, 경도
        geo = tr.find('a')['param-data'].split(',')
        info.append(geo[0])
        info.append(geo[1])
    return info

In [14]:
# 처음엔 한 두 페이지정도를 크롤링해보고 정상으로 동작하면 전체를 크롤링한다.
rent_stations = crawling_rent_station(1, 2)
# rent_stations = crawling_rent_station(1, 293)

100%|██████████| 2/2 [00:04<00:00,  2.23s/it]


In [15]:
import pandas as pd
header = ['대여소','상태','주소','위도','경도']
df = pd.DataFrame.from_records(rent_stations, columns = header)
df.shape

(2, 5)

In [16]:
df.head()

대여소   상태                              주소         위도          경도
0  롯데캐슬 115동앞  운영중     서울특별시 강동구 양재대로 1665 명일동 304  37.554920  127.142799
1  LIGA 아파트 앞  운영중  서울특별시 강동구 천호대로 1120 성내동 450-31  37.534771  127.135239

In [8]:
df.to_csv('data/bike_rent_station.csv', index=False)

In [11]:
# 파일이 제대로 생성되었는지 확인
%ls data/bike_rent_station.csv

data/bike_rent_station.csv
